# Travail de bachelor : Data processing
- Autor:   Gabriel Da Costa
- Date :   12.06.2020

In [1]:
import serial 

import sqlite3 as sql
from sqlite3 import Error
import pandas as pd

from datetime import date

## Functions for bluetooth communication

In [1]:
def connectionToBluetooth(portNumber):
    """ Establish the connection between the serial port to the bluetooth module
        with default setting.
        -------------------
        Input parameter :
        - str portNumber : name of the serial port uses for bluetooth connection (example : 'COM3')
        -------------------
        return : 
        - nothing
    """
    
    ser = serial.Serial(
            port= portNumber,
            baudrate=9600,
            parity=serial.PARITY_NONE,
            stopbits=serial.STOPBITS_ONE,
            bytesize=serial.EIGHTBITS
        )
    print(ser.name)         # check which port was really used
    ser.reset_input_buffer()
    return ser;
    
    
    
def tagsTransmission():
    """ Receive the tag numbers from the bluetooth module
        -------------------
        Input parameter :
        - nothing
        -------------------
        return : 
        - list listeTag : list of all the tag numbers received 
        - int nbrTagFound : number of tag numbers received
    """
    ## Initialisation 
    listeTag = []
    receive = True
    error = False
    ser.reset_input_buffer()

    while receive == True :     
        value = ser.read(4)
        traitable = ""

        if value == b'send': # wait for the command "send" to strat the transmission
            for i in range(0,12):
                provisoir = int(ser.read(3)) # convert to a int

                if provisoir < 15 : 
                    traitable = traitable + "0" # add 0 to the string to always have 2 bit

                traitable = traitable + str(hex(provisoir))[2:4] # convert the int value to string

            print("tag trouver :",len(listeTag)+1)

            if ser.read(3) == b'end': # wait for the command "end" to stop the transmission with succee
                print("Tag number OK")
            else:
                print("Tag number wrong")
                ser.reset_input_buffer()
                error = True
            listeTag.append(traitable) # add the tag number to the list

        elif value == b'stop':  
            receive = False
            
        elif value == b'fail':  
            # reset all parameters
            receive = True
            listeTag.clear()
            ser.reset_input_buffer()
            print("Transmission lost, restart it")

        else:   
            ser.reset_input_buffer()
            error = True
            print("Unknow data")    
            
    if error == False:
        ser.write(b'fine!') 
    else:
        ser.write(b'error') 
        print("Error, transmission failed, restart the transmission")
        
        # reset all parameters
        receive = True
        listeTag.clear()
        ser.reset_input_buffer()
        
    return listeTag;

## Establish the connection with Bluetooth serial port (COM3)

In [3]:
ser = connectionToBluetooth('COM3')

COM3


In [ ]:
listeTag = tagsTransmission()

In [1]:
print(listeTag)

In [4]:
#listeTag = []
#listeTag.append("e20000197408003612809932")

## Functions to interact with the data base Inventory

In [11]:
def connectionToDataBase(dbFile = 'Inventaire.db'):
    """ Receive the tag numbers from the bluetooth module
        -------------------
        Input parameter :
        - str dbFile : path of the data base 
        -------------------
        return : 
        - conn : data base connection variable
        - cur  : cursor for the data base utilisation
    """       
    conn = sql.connect(dbFile)
    cur = conn.cursor()
    return (conn, cur);

def createDataBase(cur,conn):
        
    Article_table = """ CREATE TABLE "Article" (
                        "tagNumber"	TEXT NOT NULL,
                        "tagFound"	INTEGER,
                        "dateFound"	TEXT NOT NULL,
                        "emplacement"	TEXT NOT NULL,
                        "locationX"	INTEGER,
                        "locationY"	INTEGER,
                        "inventaryNumber"	TEXT,
                        CONSTRAINT fk_TypeArticle
                            FOREIGN KEY (inventaryNumber)
                            REFERENCES TypeArticle(inventaryNumber)
                            ON DELETE CASCADE
                    );"""

    TypeArticle_table = """CREATE TABLE "TypeArticle" (
                            "inventaryNumber"	TEXT PRIMARY KEY,
                            "category"	TEXT,
                            "description"	TEXT
                        );"""
    
    cur.execute(TypeArticle_table)
    cur.execute(Article_table)

def changeDataBaseInfo(newValue, nameDbChange, tableDb, nameCondition, conditionValue):
    """ Receive the tag numbers from the bluetooth module
        -------------------
        Input parameter :
        - int newValue  : new value to place in the data base
        - str nameDbChange  : name of the column in the data base to change
        - str tableDb   : name of the table in the data base to change
        -------------------
        return : 
        - nothing
    """
    sql_update_query = 'UPDATE ' + tableDb + ' SET ' + nameDbChange + ' = ? WHERE ' +  nameCondition + ' = ?'
    #print(sql_update_query)
    data = (newValue, conditionValue)
    cur.execute(sql_update_query, data)
    conn.commit()
    
def initInvotaireDataBase(plotIndicator = 'Not'):
    # delete all rows from tables
    cur.execute('DELETE FROM article;',);
    cur.execute('DELETE FROM TypeArticle;',);
    
    # add data to article 
    cur.executemany('INSERT INTO article VALUES (?, ?, ?, ?, ?, ?, ?)',
            [('e20000197408003612809932', 0, '08.07.2020', 'Switzeland, Yverdon, HEIG',1 , 2, '1000'),
             ('e200001974080093128098ba', 0, '08.07.2020', 'Switzeland, Yverdon, HEIG',1 , 2, '1001'),
             ('e2000019740800331280993c', 0, '08.07.2020', 'Switzeland, Yverdon, HEIG',1 , 2, '1002'),
             ('ff2d0354740800551280990b', 0, '08.07.2020', 'Switzeland, Yverdon, HEIG',1 , 2, '1003'),
             ('e20000197408005612809914', 0, '08.07.2020', 'Switzeland, Yverdon, HEIG',1 , 2, '1004'),
             ('e20000197408003212809944', 0, '08.07.2020', 'Switzeland, Yverdon, HEIG',1 , 2, '1005'),
             ('e20000197408005412809913', 0, '08.07.2020', 'Switzeland, Yverdon, HEIG',1 , 2, '1006'),
             ('e20000197408003512809929', 0, '08.07.2020', 'Switzeland, Yverdon, HEIG',1 , 2, '1007'),
             ('e20000197408002912909531', 0, '08.07.2020', 'Switzeland, Yverdon, HEIG',1 , 2, '1008')])

    # add data to article table
    cur.executemany('INSERT INTO TypeArticle VALUES (?, ?, ?)',
            [('1000','Food','Bread 200 g'),
             ('1001','Electronics','Arduino Uno'),
             ('1002','Electronics','Arduino Uno'),
             ('1003','Electronics','Raspberry pi 3'),
             ('1004','Electronics','Raspberry pi 3'),
             ('1005','Electronics','Drone Mavic Air'),
             ('1006','HouseKeeping','Cleaner 2000'),
             ('1007','HouseKeeping','Broom Dosey'),
             ('1008','HomeAppliance','Oven PH4')])
    
    conn.commit() # save
  
    if plotIndicator == 'Not':
        return;
    
    elif plotIndicator == 'Article' or plotIndicator == 'All':
        cur.execute('SELECT * from Article')
        df_article = pd.DataFrame(cur.fetchall(), columns = ['Tag number','Tag found','Date found','Location','Sector X','Sector Y','Inventory number'])
        display(df_article)
        
    if plotIndicator == 'TypeArticle' or plotIndicator == 'All':        
        cur.execute('SELECT * from TypeArticle')
        df_typeArticle = pd.DataFrame(cur.fetchall(), columns = ['Inventory number','Category','Description'])
        display(df_typeArticle)
        
def plotTableDB(nameTable):
    cur.execute('SELECT * from ' + nameTable)
    if nameTable == 'ArticleType':
        df = pd.DataFrame(cur.fetchall(), columns = ['Inventory number','Category','Description'])
    elif nameTable == 'Article':
        df = pd.DataFrame(cur.fetchall(), columns = ['Tag number','Tag found','Date found','Location','Sector X','Sector Y','Inventory number'])
    else :
        print("ERROR : table name doesn't exist.")
        return;

    display(df)
    
def tagsFound(listTag,location,sectorX,sectorY):
    for i in range(len(listTag)):
        changeDataBaseInfo(date.today(),'dateFound' , 'Article','tagNumber',listTag[i]) #change dateFound
        changeDataBaseInfo(True,'tagFound' , 'Article','tagNumber',listTag[i])    #change tagFound
        changeDataBaseInfo(location,'emplacement' , 'Article','tagNumber',listTag[i])    #change emplacement
        changeDataBaseInfo(sectorX,'locationX' , 'Article','tagNumber',listTag[i])    #change locationX
        changeDataBaseInfo(sectorY,'locationY' , 'Article','tagNumber',listTag[i])    #change locationY
    
def resetTagFound():
    cur.execute('UPDATE Article SET tagFound = 0')
    conn.commit()

### Initialisation of the data base

In [12]:
#[conn, cur] = connectionToDataBase('InventaireTest4.db')
[conn, cur] = connectionToDataBase(r'C:\Users\Gabe\Documents\HEIG\Travail de Bachelor\Python\InventaireTest4.db')
#createDataBase(cur,conn)
initInvotaireDataBase(plotIndicator = 'All')

,Tag number,Tag found,Date found,Location,Sector X,Sector Y,Inventory number
0,e20000197408003612809932,0,08.07.2020,"Switzeland, Yverdon, HEIG",1,2,1000
1,e200001974080093128098ba,0,08.07.2020,"Switzeland, Yverdon, HEIG",1,2,1001
2,e2000019740800331280993c,0,08.07.2020,"Switzeland, Yverdon, HEIG",1,2,1002
3,ff2d0354740800551280990b,0,08.07.2020,"Switzeland, Yverdon, HEIG",1,2,1003
4,e20000197408005612809914,0,08.07.2020,"Switzeland, Yverdon, HEIG",1,2,1004
5,e20000197408003212809944,0,08.07.2020,"Switzeland, Yverdon, HEIG",1,2,1005
6,e20000197408005412809913,0,08.07.2020,"Switzeland, Yverdon, HEIG",1,2,1006
7,e20000197408003512809929,0,08.07.2020,"Switzeland, Yverdon, HEIG",1,2,1007
8,e20000197408002912909531,0,08.07.2020,"Switzeland, Yverdon, HEIG",1,2,1008


,Inventory number,Category,Description
0,1000,Food,Bread 200 g
1,1001,Electronics,Arduino Uno
2,1002,Electronics,Arduino Uno
3,1003,Electronics,Raspberry pi 3
4,1004,Electronics,Raspberry pi 3
5,1005,Electronics,Drone Mavic Air
6,1006,HouseKeeping,Cleaner 2000
7,1007,HouseKeeping,Broom Dosey
8,1008,HomeAppliance,Oven PH4


### Add the tags found in the data base Inventory

In [13]:
location = 'Switzeland, Yverdon, HEIG'
sectorX = 1
sectorY = 1
tagsFound(listeTag,location,sectorX,sectorY)
plotTableDB('Article')

,Tag number,Tag found,Date found,Location,Sector X,Sector Y,Inventory number
0,e20000197408003612809932,1,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1000
1,e200001974080093128098ba,1,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1001
2,e2000019740800331280993c,1,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1002
3,ff2d0354740800551280990b,1,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1003
4,e20000197408005612809914,1,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1004
5,e20000197408003212809944,1,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1005
6,e20000197408005412809913,1,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1006
7,e20000197408003512809929,1,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1007
8,e20000197408002912909531,1,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1008


In [14]:
resetTagFound()
plotTableDB('Article')

,Tag number,Tag found,Date found,Location,Sector X,Sector Y,Inventory number
0,e20000197408003612809932,0,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1000
1,e200001974080093128098ba,0,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1001
2,e2000019740800331280993c,0,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1002
3,ff2d0354740800551280990b,0,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1003
4,e20000197408005612809914,0,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1004
5,e20000197408003212809944,0,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1005
6,e20000197408005412809913,0,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1006
7,e20000197408003512809929,0,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1007
8,e20000197408002912909531,0,2020-07-14,"Switzeland, Yverdon, HEIG",1,1,1008


In [7]:
ser.reset_input_buffer()

In [ ]:
conn.close()             # close port
